# Tow-Based Discontinuous Composite (TBDC) property predictor
This code is written for an MSc research project at Imperial College London

## Mount drive

In [ ]:
# Data to be imported from google drive
from google.colab import drive
drive.mount('/content/drive')

# Set path for drive with datasets
drive_path ="/content/drive/MyDrive/Individual_project_ICL"

Mounted at /content/drive


## Imports and installs

In [ ]:
import sys
!{sys.executable} -m pip install tensorflow
!{sys.executable} -m pip install matplotlib
!{sys.executable} -m pip install pandas

In [ ]:
import os
import random
import time
import math
import datetime
import shutil
import json

import tensorflow as tf

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


## Import data


In [3]:
range(8)

range(0, 8)

In [ ]:
# Number of parallel jobs run on HPC specified for data loading as filenaming done based on this:
numP = 8
runName = 'TEST'

# Create empty arrays for data to be loaded:
theta_t = np.array()
Q_g = np.array()
overlap_lengths = np.array()
overlap_thicknesses = np.array()
sigma_l = np.array()
failure_factors = np.array()
theta_t2 = np.array()
failure_factors2 = np.array() # Failure factor 2 is a repeat computation of the failure factor where the only difference is another random layup sequence defined by theta_t2

for i in range(numP):
  # Read data from json file in google drive
  dataPath = os.path.join(drive_path,runName,str(i+1),'.json')

  with open(dataPath) as json_file: # load into dict
    dat = json.load(json_file)

  # Append data to numpy arrays for each parallel run to get one variable with each
  theta_t = np.append(theta_t,dat['theta_t'])
  Q_g = np.append(Q_g,dat['Q_g'])
  overlap_lengths = np.append(overlap_lengths,dat['overlap_lengths'])
  overlap_thicknesses = np.append(overlap_thicknesses,dat['overlap_thicknesses'])
  sigma_l = np.append(sigma_l,dat['sigma_l'])
  failure_factors = np.append(failure_factors,dat['failure_factors'])
  theta_t2 = np.append(theta_t2,dat['theta_t2'])
  failure_factors2 = np.append(failure_factors2,dat['failure_factors2'])


# params = list(dat.keys()) # get list of parameters in file
# print(params)



# # Read data from txt file in format "parameter:value"\n e.g. N_t:[8, 8]\n
# with open(drive_path + '/datDumpTest.json') as f:
#     dat = f.read()
# dat = dat.splitlines() # split into string array by linebreaks

# namesToLoad = ['theta_t', 'failure_factors'] # set variables to load
# datDict = {} # create dictionary to store data, for easier debugging

# for n in namesToLoad: # For each variable to be loaded
#     paramStr = [match for match in dat if n+':' in match] # find all string array entries with the given parameter name (use ":" to ensure only the actual variable is taken)
#     varname = paramStr[0].split(':')[0] # Set variable name for dictionary
#     datDict[varname] = paramStr[0].split(':')[-1] # assign key and value for parameter in dictionary
# # np.array()
# theta_t = datDict['theta_t'] # Angle of each ply in laminate
# failure_factors = datDict['failure_factors'] #

# failure_factors = np.array(list(failure_factors), dtype=float)
# print(failure_factors)

['theta_t', 'Q_g', 'overlap_lengths', 'overlap_thicknesses', 'sigma_l', 'failure_factors']
